In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df= pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df,hue="Outcome")

In [ ]:
from sklearn.model_selection import train_test_split
X=df.drop('Outcome',axis=1).values### independent features
y=df['Outcome'].values###dependent features
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=df['Outcome'])

In [ ]:
#### Create Tensors
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader


In [ ]:
X_train=torch.FloatTensor(X_train)
y_train=torch.LongTensor(y_train)
X_test=torch.FloatTensor(X_test)
y_test=torch.LongTensor(y_test)

In [ ]:
type(X_train)

In [ ]:
df.shape

In [ ]:
#### Creating Modelwith Pytorch

class ANN(nn.Module):
    def __init__(self,input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x
        

In [ ]:
##Instantiate the class
torch.manual_seed(20)
model=ANN()

In [ ]:
model.parameters

In [ ]:
## Define the bckward prop
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)


In [ ]:
y_test.shape

In [ ]:
epochs=500
final_losses=[]
for i in range(epochs):
    i+=1
    y_pred=model.forward(X_train)
    loss=loss_fn(y_pred,y_train)
    final_losses.append(loss)
    if i%10==1:
        print("Epoch num is {} and loss is {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
## Plot the loss function
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(range(epochs),final_losses)
plt.xlabel('epochs')
plt.ylabel('LOSS')

In [ ]:
### Prediciton
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_preder=model(data)
        predictions.append(y_preder.argmax().item())
        print(y_preder.argmax().item())

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)
cm

In [ ]:
plt.figure(figsize=(10,16))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [ ]:
from sklearn.metrics import r2_score
score= r2_score(y_test,predictions)

In [ ]:
score

In [ ]:
### Save the model
torch.save(model,'diabeties.pt')

In [ ]:
model=torch.load('diabeties.pt')

In [ ]:
model.eval()

In [ ]:
## predict the new data point
lii=list(df.iloc[0,:-1])

In [ ]:
lii=[6.0, 175.0, 92.0, 35.0, 0.0, 43.6, 0.727, 50.0]

In [ ]:
new_data=torch.tensor(lii)

In [ ]:
## predict new data point
with torch.no_grad():
    print(model(new_data))
    print(model(new_data).argmax().item())
    